In [3]:
# import 
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
# data load clac_nm.csv
trade_df = pd.read_csv('clac_nm.csv')

In [4]:
trade_df.de_dt = trade_df.de_dt.astype(str)
trade_df.de_tm = trade_df.de_tm.astype(str)
trade_df['date'] = trade_df.de_dt +' '+trade_df.de_tm
trade_df['date'] = trade_df['date'].map(lambda x: datetime.strptime(x,'%Y%m%d %H:%M'))

In [5]:
tmp = trade_df[['date','clnt_id','clac_nm0','buy_ct','buy_am']]
 # 결측치제거 
tmp = tmp.drop(354563)

# (buy_ct == 0개) (buy_am==0) 지우기
tmp = tmp[(tmp.buy_ct!=0)&(tmp.buy_am!=0)]

In [6]:
def cate_sp_amount(x):
    if x<=10000:
        return 'ss'
    
    elif 10000<x<=100000:
        return 'sm'
    
    elif 100000<x<=500000:
        return 'mm'
    
    elif 500000<x<=1000000:
        return 'ml'
    else:
        return 'll'

In [9]:
tmp['sp_amount'] =  tmp.buy_am /tmp.buy_ct

In [10]:
# 100만원이 넘는 것 은 중복제거하기
up_df = tmp[tmp.sp_amount>1000000]
down_df = tmp[tmp.sp_amount<=1000000]
down_df= down_df.drop_duplicates(['buy_ct','buy_am','clnt_id','clac_nm0'])
df = pd.concat([up_df,down_df],axis=0)

df['cate_sp_amount'] = df.sp_amount.apply(cate_sp_amount)

,date,clnt_id,clac_nm0,buy_ct,buy_am,sp_amount
404253,2019-07-10 16:27:00,5975,"Digiter, Appliences",1,1684000,1684000.0
510814,2019-08-14 22:21:00,33506,"Living, Health",1,2032000,2032000.0
516645,2019-09-02 23:22:00,56999,Fashion_Accessories,1,1389000,1389000.0
516660,2019-09-30 10:30:00,807,Fashion_Accessories,1,1330000,1330000.0
516661,2019-08-29 00:07:00,10237,Fashion_Accessories,1,1012000,1012000.0
...,...,...,...,...,...,...
582304,2019-09-11 13:56:00,70315,Beauty,2,3600,1800.0
582305,2019-07-13 20:53:00,40898,Beauty,1,1500,1500.0
582306,2019-07-24 12:21:00,70315,Beauty,1,6000,6000.0
582307,2019-08-12 14:14:00,63431,Beauty,1,5000,5000.0


In [12]:
df

,date,clnt_id,clac_nm0,buy_ct,buy_am,sp_amount,cate_sp_amount
404253,2019-07-10 16:27:00,5975,"Digiter, Appliences",1,1684000,1684000.0,ll
510814,2019-08-14 22:21:00,33506,"Living, Health",1,2032000,2032000.0,ll
516645,2019-09-02 23:22:00,56999,Fashion_Accessories,1,1389000,1389000.0,ll
516660,2019-09-30 10:30:00,807,Fashion_Accessories,1,1330000,1330000.0,ll
516661,2019-08-29 00:07:00,10237,Fashion_Accessories,1,1012000,1012000.0,ll
...,...,...,...,...,...,...,...
582304,2019-09-11 13:56:00,70315,Beauty,2,3600,1800.0,ss
582305,2019-07-13 20:53:00,40898,Beauty,1,1500,1500.0,ss
582306,2019-07-24 12:21:00,70315,Beauty,1,6000,6000.0,ss
582307,2019-08-12 14:14:00,63431,Beauty,1,5000,5000.0,ss


## main_category

In [16]:
idx2 = df.groupby('clnt_id')['clac_nm0'].value_counts().unstack().fillna(0)
idx2['main_category'] = list(map(max,idx2.values))
idx2

clac_nm0,Baby,Beauty,"Digiter, Appliences",Fashion_Accessories,Fashion_Clothes,Foods,Interier,"Living, Health",Others,"Sports, Leisure","Travel, Culture",main_category
clnt_id,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0
9,0.0,1.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0
12,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
20,0.0,0.0,0.0,0.0,0.0,33.0,0.0,1.0,0.0,0.0,0.0,33.0
23,0.0,0.0,0.0,0.0,1.0,35.0,1.0,0.0,0.0,0.0,0.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72373,0.0,3.0,1.0,0.0,0.0,9.0,2.0,2.0,2.0,0.0,0.0,9.0
72400,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0
72410,0.0,1.0,0.0,0.0,3.0,39.0,0.0,2.0,0.0,0.0,0.0,39.0


In [46]:
def match_max_clac_nm0(x):
    if x[0] == x[11]:
        return 'Baby'
    elif x[1] == x[11]:
        return 'Beauty'
    elif x[2] == x[11]:
        return 'Digiter, Appliences'
    elif x[3] == x[11]:
        return 'Fashion_Accessories'
    elif x[4] == x[11]:
        return 'Fashion_Clothes'
    elif x[5] == x[11]:
        return 'Foods'
    elif x[6] == x[11]:
        return 'Interier'
    elif x[7] == x[11]:
        return 'Living, Health'
    elif x[8] == x[11]:
        return 'Others'
    elif x[9] == x[11]:
        return 'Sports, Leisure'
    elif x[10] == x[11]:
        return 'Travel, Culture'



In [51]:
idx2['main_category_value'] = \
   idx2[['Baby', 'Beauty', 'Digiter, Appliences', 'Fashion_Accessories',\
       'Fashion_Clothes', 'Foods', 'Interier', 'Living, Health', 'Others',\
       'Sports, Leisure', 'Travel, Culture', 'main_category']].apply(match_max_clac_nm0,axis=1)

In [52]:
idx2

clac_nm0,Baby,Beauty,"Digiter, Appliences",Fashion_Accessories,Fashion_Clothes,Foods,Interier,"Living, Health",Others,"Sports, Leisure","Travel, Culture",main_category,_match_max_clac_nm0_value,main_category_value
clnt_id,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,Foods,Foods
9,0.0,1.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0,Foods,Foods
12,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,Foods,Foods
20,0.0,0.0,0.0,0.0,0.0,33.0,0.0,1.0,0.0,0.0,0.0,33.0,Foods,Foods
23,0.0,0.0,0.0,0.0,1.0,35.0,1.0,0.0,0.0,0.0,0.0,35.0,Foods,Foods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72373,0.0,3.0,1.0,0.0,0.0,9.0,2.0,2.0,2.0,0.0,0.0,9.0,Foods,Foods
72400,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,Beauty,Beauty
72410,0.0,1.0,0.0,0.0,3.0,39.0,0.0,2.0,0.0,0.0,0.0,39.0,Foods,Foods


In [53]:
aa = idx2.reset_index().sort_values('clnt_id')

In [55]:
aa[['clnt_id','main_category_value']]

clac_nm0,clnt_id,main_category_value
0,2,Foods
1,9,Foods
2,12,Foods
3,20,Foods
4,23,Foods
...,...,...
11261,72373,Foods
11262,72400,Beauty
11263,72410,Foods
11264,72423,"Digiter, Appliences"


# sp_amount, various_count
- sp_amount == (구매 같은 개수 카운트) :얼마나 동일한 상품을 사는지 
- various_count == clac_nm0.unique()/len(clac_nm0):얼마나 다양한 상품군을 사는지

In [13]:
sp_amount_count = []
various_count = []


for i in list(df.clnt_id.unique()):
    tmp = df[df.clnt_id == i]
    
    sp_amount_count.append(len(tmp.sp_amount.unique()))
    various_count.append(len(tmp.clac_nm0.unique())/len(tmp.clac_nm0))

f_0_1 = pd.DataFrame()
f_0_1['clnt_id'] = list(df.clnt_id.unique())
f_0_1['sp_amount_count'] = sp_amount_count
f_0_1['various_count'] = various_count
f_0_1

,clnt_id,sp_amount_count,various_count
0,5975,53,0.185185
1,33506,28,0.103448
2,56999,14,0.357143
3,807,155,0.041667
4,10237,1,1.000000
...,...,...,...
11261,53632,1,1.000000
11262,7957,1,1.000000
11263,58443,1,1.000000
11264,21232,1,1.000000


In [33]:
f_0_1 = f_0_1.sort_values('clnt_id')

In [57]:
f_0_1['main_category'] = list(idx2['main_category_value'])

In [58]:
f_0_1

,clnt_id,sp_amount_count,various_count,main_category
7895,2,9,0.166667,Foods
1640,9,26,0.076923,Foods
8262,12,1,1.000000,Foods
6121,20,33,0.058824,Foods
2456,23,36,0.081081,Foods
...,...,...,...,...
4459,72373,18,0.315789,Foods
8821,72400,4,0.750000,Beauty
1732,72410,43,0.088889,Foods
10411,72423,2,1.000000,"Digiter, Appliences"


In [59]:
f_0_1.to_csv('선호상품군.csv')